In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        
        self.fc1 = nn.Linear(16*6*6,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()
        num_features = 1
        for s in size:
            num_features *=s
        return num_features

In [6]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
parem = list(net.parameters())
print(len(parem))
print(parem)
print(parem[0].size())

10
[Parameter containing:
tensor([[[[-0.2021, -0.2799, -0.0432],
          [-0.2389, -0.2692,  0.0565],
          [-0.2069, -0.0780, -0.2091]]],


        [[[ 0.3166,  0.2242, -0.1714],
          [ 0.2376,  0.1784, -0.2354],
          [-0.2686,  0.0643,  0.1287]]],


        [[[-0.2212, -0.0908, -0.1518],
          [ 0.2048,  0.0976,  0.0360],
          [-0.2019, -0.2222,  0.2718]]],


        [[[ 0.3170, -0.2135,  0.2688],
          [-0.0221,  0.1228,  0.2263],
          [ 0.1154,  0.0820,  0.1316]]],


        [[[-0.2642,  0.0575,  0.3262],
          [ 0.1129, -0.2077, -0.2350],
          [ 0.2415,  0.2606,  0.0346]]],


        [[[ 0.1690, -0.3219, -0.0939],
          [-0.1196,  0.0782,  0.2793],
          [ 0.0166, -0.2357, -0.0325]]]], requires_grad=True), Parameter containing:
tensor([-0.1388,  0.0957,  0.0845,  0.2073, -0.3131,  0.2854],
       requires_grad=True), Parameter containing:
tensor([[[[ 0.0514, -0.1129,  0.1005],
          [-0.0558,  0.0948,  0.0564],
          [ 0.0

In [13]:
input = torch.randn(1, 1, 32, 32)
print("size is ",input.size())
out = net(input)
print(out)
print(out.size())

size is  torch.Size([1, 1, 32, 32])
tensor([[-0.0337, -0.0205, -0.1069,  0.0394, -0.0584,  0.0853,  0.0398,  0.1355,
          0.1100,  0.1521]], grad_fn=<AddmmBackward>)
torch.Size([1, 10])


In [9]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [10]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7099, grad_fn=<MseLossBackward>)


In [11]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [17]:
net.zero_grad()
print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)
loss.backward()
print("conv1.bias.grad after backward")
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0070, -0.0043,  0.0075, -0.0071,  0.0142, -0.0058])


In [18]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [19]:

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

NameError: name 'optim' is not defined